**Unleashing the Power of LangChain Expression Language (LCEL): from proof of concept to production**

In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [3]:
!jupyter nbconvert --execute index.ipynb

[NbConvertApp] WARNING | pattern 'index.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execu

**Example — Company name & Business model with Old Chains**

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI

USER_INPUT = "colorful socks"
llm = OpenAI(temperature=0)

prompt_template_product = "What is a good name for a company that makes {product}?"
company_name_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template_product))
company_name_output = company_name_chain(USER_INPUT)

prompt_template_business = "Give me the best business model idea for my company named: {company}"
business_model_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template_business))
business_model_output = business_model_chain(company_name_output["text"])

print(company_name_output)
print(business_model_output)

In [5]:
>>> {'product': 'colorful socks', 'text': 'Socktastic!'}
>>> {'company': 'Socktastic!','text': "A subscription-based service offering a monthly delivery..."}

{'company': 'Socktastic!',
 'text': 'A subscription-based service offering a monthly delivery...'}

**Example — Customization & Routing with Old Chains**

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI
import ast


USER_INPUT = "Harrison Chase"
llm = OpenAI(temperature=0)

# ---- Same code as before
prompt_template_product = "What is a good name for a company that makes {product}?"
company_name_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template_product))

prompt_template_business = "Give me the best business model idea for my company named: {company}"
business_model_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template_business))

# ---- New code

prompt_template_is_product = (
    "Your goal is to find if the input of the user is a plausible product name\n"
    "Questions, greetings, long sentences, celebrities or other non relevant inputs are not considered products\n"
    "input: {product}\n"
    "Answer only by 'True' or 'False' and nothing more\n"
)

prompt_template_cannot_respond = (
    "You cannot respond to the user input: {product}\n"
    "Ask the user to input the name of a product in order for you to make a company out of it.\n"
)

cannot_respond_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template_cannot_respond))
company_name_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template_product))
business_model_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template_business))
is_a_product_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template_is_product))

# If we use bool on a non empty str it will be True, so we need `literal_eval`
is_a_product = ast.literal_eval(is_a_product_chain(USER_INPUT)["text"])
if is_a_product:
    company_name_output = company_name_chain(USER_INPUT)
    business_model_output = business_model_chain(company_name_output["text"])
    print(business_model_output)
else:
    print(cannot_respond_chain(USER_INPUT))

**Example — Company name & Business model with LCEL**

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI

USER_INPUT = "colorful socks"
llm = OpenAI(temperature=0)

prompt_template_product = "What is a good name for a company that makes {product}?"
prompt_template_business = "Give me the best business model idea for my company named: {company}"

chain = (
  PromptTemplate.from_template(prompt_template_product)
  | llm
  | {'company': RunnablePassthrough()}
  | PromptTemplate.from_template(prompt_template_business)
  | llm
)

business_model_output = chain.invoke({'product': 'colorful socks'})

**Example — Customisation & Routing with LCEL**

In [ ]:
from typing import Dict
from langchain_core.runnables import RunnablePassthrough, RunnableBranch
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import BooleanOutputParser
from langchain_community.llms import OpenAI

USER_INPUT = "Harrrison Chase"
llm = OpenAI(temperature=0)

prompt_template_product = "What is a good name for a company that makes {product}?"
prompt_template_cannot_respond = (
    "You cannot respond to the user input: {product}\n"
    "Ask the user to input the name of a product in order for you to make a company out of it.\n"
)
prompt_template_business = "Give me the best business model idea for my company named: {company}"
prompt_template_is_product = (
    "Your goal is to find if the input of the user is a plausible product name\n"
    "Questions, greetings, long sentences, celebrities or other non relevant inputs are not considered products\n"
    "input: {product}\n"
    "Answer only by 'True' or 'False' and nothing more\n"
)

answer_user_chain = (
    PromptTemplate.from_template(prompt_template_product)
  | llm
  | {'company': RunnablePassthrough()}
  | PromptTemplate.from_template(prompt_template_business)
  | llm
).with_types(input_type=Dict[str, str], output_type=str)

is_product_chain = (
    PromptTemplate.from_template(prompt_template_is_product)
    | llm
    | BooleanOutputParser(true_val='True', false_val='False')
    ).with_types(input_type=Dict[str, str], output_type=bool)

cannot_respond_chain = (
    PromptTemplate.from_template(prompt_template_cannot_respond) | llm
    ).with_types(input_type=Dict[str, str], output_type=str)

full_chain = RunnableBranch(
    (is_product_chain, answer_user_chain),
    cannot_respond_chain
).with_types(input_type=Dict[str, str], output_type=str)

print(full_chain.invoke({'product': USER_INPUT}))

**The Runnable interface**

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI

chain_number_one = (
  PromptTemplate.from_template(prompt_template_product)
  | llm
  | {'company': RunnablePassthrough()} # <--- THIS WILL CHANGE
  | PromptTemplate.from_template(prompt_template_business)
  | llm
)

chain_number_two = (
  PromptTemplate.from_template(prompt_template_product)
  | llm
  | RunnableParallel(company=RunnablePassthrough()) # <--- THIS CHANGED
  | PromptTemplate.from_template(prompt_template_business)
  | llm
)

print(chain_number_one == chain_number_two)
>>> True